# MatplotlibとSeabornの使い方

可視化は「数表だけでは見えない傾向」を発見するための手段です。
Matplotlib は柔軟な描画基盤、Seaborn は統計可視化を短いコードで書くための拡張です。

このノートでは、折れ線・散布図・ヒストグラム・カテゴリ比較を実データ風の例で確認します。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", context="notebook")


まずは折れ線グラフ。時系列の増減を見るときに使います。

In [ ]:
days = np.arange(1, 11)
loss = np.array([1.8, 1.5, 1.3, 1.15, 1.05, 0.96, 0.90, 0.86, 0.82, 0.79])

fig, ax = plt.subplots(figsize=(6, 3.5))
ax.plot(days, loss, marker="o", linewidth=2)
ax.set_title("Training Loss")
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
plt.show()


散布図は、2 変数の関係を見るときの第一選択です。
ここでは「学習時間が増えると点数が上がる」という仮想データを作るために、
`uniform` で学習時間、`normal` でノイズを発生させています。

In [ ]:
rng = np.random.default_rng(7)
hours = rng.uniform(1, 8, size=80)
score = 52 + hours * 6 + rng.normal(0, 6, size=80)

fig, ax = plt.subplots(figsize=(5.5, 4))
ax.scatter(hours, score, alpha=0.75)
ax.set_title("Study Hours vs Score")
ax.set_xlabel("Hours")
ax.set_ylabel("Score")
plt.show()


ヒストグラムは分布の形を確認するために使います。ビン数を変えると見え方が変わる点も重要です。

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
axes[0].hist(score, bins=8, color="#4C78A8", alpha=0.85)
axes[0].set_title("bins=8")
axes[1].hist(score, bins=20, color="#F58518", alpha=0.85)
axes[1].set_title("bins=20")
for ax in axes:
    ax.set_xlabel("Score")
axes[0].set_ylabel("Count")
plt.tight_layout()
plt.show()


カテゴリ比較では、箱ひげ図や棒グラフが便利です。Seaborn はこの部分を短く書けます。

この例では `np.repeat` でクラスラベルを増やし、
`np.concatenate` でクラスごとに分布の違うスコア列を連結して、
「クラスごとのばらつき」を比較できる形にしています。

In [ ]:
df = pd.DataFrame({
    "class": np.repeat(["A", "B", "C"], repeats=30),
    "score": np.concatenate([
        rng.normal(70, 7, size=30),
        rng.normal(78, 6, size=30),
        rng.normal(74, 8, size=30),
    ])
})

fig, ax = plt.subplots(figsize=(6, 4))
sns.boxplot(data=df, x="class", y="score", ax=ax)
ax.set_title("Score Distribution by Class")
plt.show()


複数系列の比較は、`hue` を使うと情報を整理しやすくなります。
この例では baseline と improved を同じ軸で比較し、
improved の曲線がより早く高い精度に到達しているかを読むのがポイントです。

In [ ]:
df2 = pd.DataFrame({
    "week": np.tile(np.arange(1, 7), 2),
    "accuracy": np.concatenate([
        np.array([0.61, 0.66, 0.70, 0.74, 0.77, 0.79]),
        np.array([0.58, 0.64, 0.69, 0.72, 0.75, 0.78]),
    ]),
    "model": ["baseline"] * 6 + ["improved"] * 6,
})

fig, ax = plt.subplots(figsize=(6, 3.8))
sns.lineplot(data=df2, x="week", y="accuracy", hue="model", marker="o", ax=ax)
ax.set_ylim(0.55, 0.82)
ax.set_title("Validation Accuracy")
plt.show()


相関行列ヒートマップは、特徴量同士の関係を俯瞰するときに便利です。

In [ ]:
feature_df = pd.DataFrame({
    "math": rng.normal(75, 10, size=120),
    "english": rng.normal(73, 9, size=120),
    "science": rng.normal(70, 11, size=120),
    "study_hours": rng.normal(4.5, 1.2, size=120),
})

corr = feature_df.corr(numeric_only=True)
fig, ax = plt.subplots(figsize=(5, 4.2))
sns.heatmap(corr, annot=True, cmap="Blues", vmin=-1, vmax=1, ax=ax)
ax.set_title("Correlation Matrix")
plt.show()


可視化では、見た目を整える前に「何を比較したいか」を先に決めることが重要です。
目的が定まると、グラフ種類・軸・色分けの選択がぶれにくくなります。